In [4]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mlp
import random
from pathlib import Path
import os
import glob
from operator import itemgetter

import cv2
import tensorflow.experimental.numpy as tnp
import numpy as np
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import asarray
from numpy.random import randint
from numpy import savez_compressed
from numpy import load 
from numpy.random import randn


import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import Conv2DTranspose, MaxPooling2D, UpSampling2D
from tensorflow.keras.layers import Reshape
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img 
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dropout 
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam, Adagrad
from tensorflow.keras.utils import plot_model
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
#from tensorflow.keras.applications import VGG16

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import CSVLogger
from tqdm import tqdm

#backend.set_image_dim_ordering('th')
#backend.image_data_format()

# Load .npz files

In [20]:
npy_file = 'E:/84 paper/'
npy_file = os.listdir(npy_file)

In [21]:
npy_file

['Abuse001_x264_i3d.npy',
 'Abuse001_x264_i3d_labels.npy',
 'Abuse002_x264_i3d.npy',
 'OneDrive_2021-08-10.zip']

In [23]:
npy_files = np.load('E:/84 paper/Abuse001_x264_i3d_labels.npy')

In [24]:
npy_files.shape

(171, 10, 2048)

In [3]:
os.getcwd()

'E:\\'

In [4]:
# #### combining the npz
npz_path = "E:/UCF_Crimes/npz_120"
npz_list = os.listdir(npz_path)

In [5]:
npz_list

['crimes_120_0.npz',
 'crimes_120_1.npz',
 'crimes_120_10.npz',
 'crimes_120_11.npz',
 'crimes_120_12.npz',
 'crimes_120_2.npz',
 'crimes_120_3.npz',
 'crimes_120_4.npz',
 'crimes_120_5.npz',
 'crimes_120_6.npz',
 'crimes_120_7.npz',
 'crimes_120_8.npz',
 'crimes_120_9.npz',
 'crimes_1_25570.npz',
 'crimes_1_25571.npz']

In [6]:
full_npz = itemgetter(0,1,2,7)(npz_list)
full_npz

('crimes_120_0.npz',
 'crimes_120_1.npz',
 'crimes_120_10.npz',
 'crimes_120_4.npz')

In [7]:
# full_npz_val = itemgetter(4, 9)(npz_list)
# full_npz_val

In [8]:
full_npz_test = itemgetter(13, 14)(npz_list)
full_npz_test

('crimes_1_25570.npz', 'crimes_1_25571.npz')

In [9]:
full_npz = [npz_path+ '/' + npz for npz in full_npz]
full_npz

['E:/UCF_Crimes/npz_120/crimes_120_0.npz',
 'E:/UCF_Crimes/npz_120/crimes_120_1.npz',
 'E:/UCF_Crimes/npz_120/crimes_120_10.npz',
 'E:/UCF_Crimes/npz_120/crimes_120_4.npz']

In [10]:
# full_npz_val = [npz_path+ '/' + npz for npz in full_npz_val]
# full_npz_val

In [11]:
full_npz_test = [npz_path + '/' + npz for npz in full_npz_test]
full_npz_test

['E:/UCF_Crimes/npz_120/crimes_1_25570.npz',
 'E:/UCF_Crimes/npz_120/crimes_1_25571.npz']

In [12]:
data_all = [np.load(fname) for fname in full_npz] 

In [13]:
# data_all_val = [np.load(fname) for fname in full_npz_val] 

In [14]:
data_test = [np.load(fname) for fname in full_npz_test]

In [15]:
images = list()
images += [npz['arr_0'] for npz in data_all]

In [16]:
labels = list()
labels += [npz['arr_1'] for npz in data_all]

In [17]:
# images_val = list()
# images_val += [npz['arr_0'] for npz in data_all_val]

In [18]:
# labels_val = list()
# labels_val += [npz['arr_1'] for npz in data_all_val]

In [19]:
images_test = list()
images_test += [npz['arr_0'] for npz in data_test]

In [20]:
labels_test = list()
labels_test += [npz['arr_1'] for npz in data_test]

## Light Augmenter 

In [21]:
def adjust_gamma(img):
    ratio = 0.5 / 1
    if ratio >= 1:
        print("Image already bright enough")
        return img

    # Otherwise, adjust brightness to get the target brightness
    return cv2.convertScaleAbs(img, alpha = 1 / ratio, beta = 0)

In [22]:
def blur(image):
    return cv2.blur(image,(5,5))

In [23]:
class CustomAugmentation(object):
    """ Defines a custom augmentation class"""
    
    kernel = np.ones((3,3),np.uint8)
    
    def __init__(self, erosion = False, dilation = False, light = False,
                       sharpness = False, blur = False):
        self.erosion = erosion
        self.dilation = dilation
        self.light = light
        self.sharpness = sharpness
        self.blur = blur
    
    def __call__(self, img):
        
        randomNumber = np.random.random()
        
        # Erosion and dilation are never applied together
        
        if randomNumber < 0.9:
            pass
        elif randomNumber < 0.95:
            if self.erosion == True:
                # Apply erosion 5% of the time if True
                img = cv2.erode(img,CustomAugmentation.kernel,iterations = 1)
                img = img.reshape(120,120,3)
                
        elif self.dilation == True:
                # Apply dilation 5% of the time if True
            img = cv2.dilate(img, CustomAugmentation.kernel,iterations = 1)
            img = img.reshape(120,120,3)
                
        elif self.light == True:
            img = adjust_gamma(img)
            img = img.reshape(120,120,3)

        elif self.sharpness == True:
            kernel = np.array([[0, -1, 0], [-1, 5,-1], [0, -1, 0]])
            img = cv2.filter2D(img, -1, kernel)
            img = img.reshape(120, 120, 3)
            
        elif self.blur == True:
            img = blur(img) if np.random.random()>0.5 else add_sharpness(img)
            img = img.reshape(120, 120, 3)
                
        return img

# Before all, define a custom activation function

In [24]:
# custom activation function
def custom_activation(output):
    logexpsum = backend.sum(backend.exp(output), axis=-1, keepdims=True)
    result = logexpsum / (logexpsum + 1.0)
    return result

# Defining the models.

## Discriminator (standalone supervised and unsupervised discriminator models)

In [25]:
# define the standalone supervised and unsupervised discriminator models
def define_discriminator(in_shape=(120, 120, 3), n_classes=14):
    init = RandomNormal(stddev=0.02)
# image input
    in_image = Input(shape=in_shape)

# #downsample
    c1 = Conv2D(64, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(in_image)
    c1 = LeakyReLU(alpha=0.4)(c1)
    c1 = BatchNormalization(axis=1)(c1)
    
    c1 = Conv2D(64, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(c1)
    c1 = LeakyReLU(alpha=0.4)(c1)
    c1 = BatchNormalization(axis=1)(c1)
    
    c1 = MaxPooling2D(pool_size = (2,2))(c1)
    c1 = Dropout(0.2)(c1)
    
    c1 = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(c1)
    c1 = LeakyReLU(alpha=0.4)(c1)
    c1 = BatchNormalization(axis=1)(c1)
    
    c1 = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(c1)
    c1 = LeakyReLU(alpha=0.4)(c1)
    c1 = BatchNormalization(axis=1)(c1)
    
    c1 = MaxPooling2D(pool_size = (2,2))(c1)
    c1 = Dropout(0.2)(c1)
    
    c1 = Conv2D(256, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(c1)
    c1 = LeakyReLU(alpha=0.4)(c1)
    c1 = BatchNormalization(axis=1)(c1)
    
    c1 = Conv2D(256, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(c1)
    c1 = LeakyReLU(alpha=0.4)(c1)
    c1 = BatchNormalization(axis=1)(c1)
    
#     c1 = MaxPooling2D(pool_size = (2,2))(c1)
    c1 = Dropout(0.2)(c1)
    
# flatten feature maps
    fe = Flatten()(c1)
#     fe = (Dense(256))(fe)
#     fe = LeakyReLU(alpha=0.4)(fe)
#     fe = BatchNormalization(axis=1)(fe)
# dropout
    fe = Dropout(0.2)(fe)
    
# output layer nodes
    fe = Dense(n_classes, kernel_initializer=init)(fe)
# supervised output
    c_out_layer = Activation('softmax')(fe)
# define and compile supervised discriminator model
    c_model = Model(in_image, c_out_layer)
    c_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adagrad(lr=0.0005), metrics=['accuracy']) #beta_1=0.5
# unsupervised output
    d_out_layer = Lambda(custom_activation)(fe)
# define and compile unsupervised discriminator model
    d_model = Model(in_image, d_out_layer)
    d_model.compile(loss='mse', optimizer=Adam(lr=0.0002, beta_1=0.5))
    return d_model, c_model

## Classifier (Define the standalone classifier model)

In [26]:
def define_standalone_classifier(in_shape=(120,120,3), n_classes=14):
    # image input
    in_image = Input(shape=in_shape)
    # Conv Layer and downsample
    fe = Conv2D(32, (3,3))(in_image)
    fe = LeakyReLU(alpha=0)(fe)
    fe = BatchNormalization(axis=1)(fe)
    fe = MaxPooling2D(pool_size = (2,2))(fe)
   # Conv Layer and downsample
    fe = Conv2D(32, (3,3))(fe)
    fe = LeakyReLU(alpha=0)(fe)
    fe = MaxPooling2D(pool_size = (2,2))(fe)
    # Conv Layer and downsample
    fe = Conv2D(64, (3,3))(fe)
    fe = LeakyReLU(alpha=0)(fe)
    fe = BatchNormalization(axis=1)(fe)
    fe = MaxPooling2D(pool_size = (2,2))(fe)
    # Conv Layer and downsample
    fe = Conv2D(128, (3,3))(fe)
    fe = LeakyReLU(alpha=0)(fe)
    fe = MaxPooling2D(pool_size = (2,2))(fe)
    # flatten feature maps
    fe = Flatten()(fe)
    # dropout
    fe = Dense(128)(fe)
    fe = LeakyReLU(alpha=0)(fe)
    fe = Dropout(0.5)(fe)
    # output layer nodes
    fe = Dense(n_classes)(fe)
    # supervised output
    c_out_layer = Activation('sigmoid')(fe)
    # define and compile standalone classifier model
    c_model = Model(in_image, c_out_layer)
    c_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return c_model

##  Generator

In [27]:
# define the standalone generator model
def define_generator(latent_dim):
    init = RandomNormal(stddev=0.02)
    #image generator input
    in_lat = Input(shape=(latent_dim,))
# foundation for 120x120 image (the 15*15 is the 1/8 of the size of the image. metric given by the book)
    n_nodes = 128 * 15 * 15
    gen = Dense(n_nodes)(in_lat)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Reshape((15, 15, 128))(gen)
    # upsample to 30x30
    gen = Conv2DTranspose(32, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(gen)
    gen = BatchNormalization(axis=1)(gen)
    gen = LeakyReLU(alpha=0.4)(gen)
#upsample to 60x60
    gen = Conv2DTranspose(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(gen)
    gen = BatchNormalization(axis=1)(gen)
    gen = LeakyReLU(alpha=0.4)(gen)
# #upsample to 120x120
    gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(gen)
    gen = BatchNormalization(axis=1)(gen)
    gen = LeakyReLU(alpha=0.4)(gen)
# output
    out_layer = Conv2D(3, (15, 15), activation='tanh', padding='same', kernel_initializer=init)(gen)
# define model 
    model = Model(in_lat, out_layer)
    return model

### combined generator and discriminator model (for updating the generator) 

In [28]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
# make weights in the discriminator not trainable
    d_model.trainable = False
# connect image output from generator as input to discriminator
    gan_output = d_model(g_model.output)
# define gan model as taking noise and outputting a classification
    model = Model(g_model.input, gan_output)
# compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='mse', optimizer=opt)
    return model

# select a supervised subset of the dataset

In [29]:
def select_supervised_samples(dataset, n_samples=5000, n_classes=14):
    X, y = dataset
    X_list, y_list = list(), list()
    n_per_class = int(n_samples / n_classes)
    for n in range (0, 4):
        for i in range(n_classes):
            # get all images for this class
            X_with_class = [X[n]][[y[n]] == i]
            # choose random instances
            ix = randint(0, len(X_with_class), n_per_class)
            # add to list
            [X_list.append(X_with_class[j]) for j in ix]
            [y_list.append(i) for j in ix]
    return asarray(X_list), asarray(y_list) 

In [30]:
def select_supervised_samples2(dataset, n_samples=5000, n_classes=14):
    X, y = dataset
    X_list, y_list = list(), list()
    n_per_class = int(n_samples / n_classes)
    for n in range (0, 2):
        for i in range(n_classes):
            # get all images for this class
            X_with_class = [X[n]][[y[n]] == i]
            # choose random instances
            ix = randint(0, len(X_with_class), n_per_class)
            # add to list
            [X_list.append(X_with_class[j]) for j in ix]
            [y_list.append(i) for j in ix]
    return asarray(X_list), asarray(y_list) 

# Select random samples from the supervised dataset

In [31]:
def generate_real_samples(X, y, n_samples):
# split into images and labels
    ima = X
    lab = y
# choose random instances
    ix = randint(0, ima.shape[0], n_samples)
# select images and labels
    X, labels = ima[ix], lab[ix]
# generate class labels
    y = ones((n_samples, 1))
    return [X, lab], y

In [32]:
def generate_real_samples2(x, y, n_samples):
# split into images and labels
    ix = randint(0, x[0].shape[0], n_samples)
    im_list = list()
    lab_list = list()
# select images and labels
    for n in range (0, 4): 
        for i in x[n][ix]:
            im_list.append(i)
        for c in y[n][ix]:
            lab_list.append(c)
# generate class labels
    y = ones((n_samples*4, 1))
    return [asarray(im_list), asarray(lab_list)], y

## generate points in latent space

In [33]:
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	z_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	z_input = z_input.reshape(n_samples, latent_dim)
	return z_input

## Generate the fake images

In [34]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
# generate points in latent space
    z_input = generate_latent_points(latent_dim, n_samples)
# predict outputs
    images = generator.predict(z_input)
# create class labels
    y = zeros((n_samples, 1))
    return images, y

# save as a plot and save the model

In [35]:
def summarize_performance(step, g_model, c_model, latent_dim, dataset2, max_acc_c,folder, n_samples=100):
    
    # prepare fake examples
    X, _ = generate_fake_samples(g_model, latent_dim, n_samples)
    # scale from [-1,1] to [0,1]
    X = (X + 1) / 2.0
    # plot images
    for i in range(9):
        # define subplot
        plt.subplot(3, 3, 1 + i)
        # turn off axis
        plt.axis('off')
        # plot raw pixel data
        plt.imshow(X[i, :, :, 0])
    # save plot to file
    filename1 = folder+'/generated_plot_%04d.png' % (step+1)
    plt.savefig(filename1)
    plt.close()
    # evaluate the classifier model
    X, y = dataset2
    
#     for n in range(0, 4):
    for n in range(0,2):
        _, acc = c_model.evaluate(X[n], y[n], verbose=0)
        if acc>max_acc_c:
            print('Best acc so far! best_acc_c = %.3f%%' % (acc * 300))           
            max_acc_c = acc
            c_model.save(folder+'/c_model_best.h5')
        else:
            print('Best acc did not improve! best_acc_c = %.3f%%' % (max_acc_c * 300))
        
    #print('Classifier Accuracy: %.3f%%' % (acc * 100))
    # save the generator model
    filename2 = folder+'/g_model_%04d.h5' % (step+1)
    g_model.save(filename2)
    # save the classifier model
    filename3 = folder+'/c_model_%04d.h5' % (step+1)
    c_model.save(filename3)
    print('>Saved: %s, %s, and %s' % (filename1, filename2, filename3))  
    
    return max_acc_c   


# Training the models

In [36]:
earlystop = EarlyStopping(monitor = 'accuracy', 
                          min_delta = 0, 
                          patience = 5,
                          verbose = 1,
                          restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'accuracy',
                              factor = 0.2,
                              patience = 6,
                              verbose = 1,
                              min_delta = 0.0001)

# we put our call backs into a callback list
callbacks = [earlystop, reduce_lr]

In [37]:
preprocesor = CustomAugmentation(erosion=False, dilation= False, light= True, sharpness = False, blur = False)

In [38]:
def train(g_model, d_model, c_model, gan_model, dataset, latent_dim,folder,n_samples, n_epochs=50, n_batch=100, with_aug=True):
    
    max_acc_c =0
    # select supervised dataset
    X_sup, y_sup = select_supervised_samples(dataset,n_samples)
    
    X_val, y_val = select_supervised_samples2(dataset,n_samples)
    
    print(X_sup.shape, y_sup.shape)
    # calculate the number of batches per training epoch
    bat_per_epo = int(dataset[0][0].shape[0] / n_batch)
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    # calculate the size of half a batch of samples
    half_batch = int(n_batch / 2)
    
    if(with_aug):
        train_datagen = ImageDataGenerator( preprocessing_function = preprocesor) #rotation_range = 10,
                             #zoom_range = 0.3, width_shift_range = 0.2, height_shift_range = 0.2 )

    else:
        train_datagen = ImageDataGenerator(
                horizontal_flip=False)  
        
        
    print('n_epochs=%d, n_batch=%d, 1/2=%d, b/e=%d, steps=%d' % (n_epochs, n_batch, half_batch, bat_per_epo, n_steps))
    # manually enumerate epochs, get one 
    for i in range(n_epochs):
        print("Epoch {}/{}".format(i, n_epochs))
        flow_iter = train_datagen.flow(X_sup, y_sup, half_batch) #this is an enumerator for random half-batches
        with tqdm(total = bat_per_epo, position = 0, leave=True) as progress_bar:
            for j, [Xsup_real, ysup_real] in enumerate(flow_iter):
                if j >= bat_per_epo:
                    break #the enumerator itself will run forever, stop on expected number of batches
                progress_bar.update(1)
                
                c_model.fit(Xsup_real, ysup_real, validation_data = (X_val, y_val), callbacks = callbacks, verbose=0)
                
                # update unsupervised discriminator (d)
                [X_real, _], y_real = generate_real_samples2(images, labels, half_batch) #generate real, but not labeled samples for the discriminator
                d_model.train_on_batch(X_real, y_real)
                
                X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
                d_model.train_on_batch(X_fake, y_fake)
                
                # update generator (g)
                X_gan, y_gan = generate_latent_points(latent_dim, n_batch), ones((n_batch, 1))
                gan_model.train_on_batch(X_gan, y_gan)

        max_acc_c= summarize_performance(i, g_model, c_model, latent_dim, dataset2, max_acc_c, folder)

In [39]:
# # train the standalone classifier
# def train_stand_alone_c(c_model, dataset, folder, n_epochs=50, n_batch=100, n_samples=100,n_classes=14,with_aug=True):
#     max_acc_c = 0
#     # select supervised dataset
#     X_sup, y_sup = select_supervised_samples(dataset,n_samples,n_classes)
    
#     print(X_sup.shape, y_sup.shape)
#     # calculate the number of batches per training epoch
#     bat_per_epo = int(X_sup.shape[0] / n_batch)
#     # calculate the number of training iterations
#     n_steps = bat_per_epo * n_epochs
#     # calculate the size of half a batch of samples
#     half_batch = int(n_batch / 2)
#     print('n_epochs=%d, n_batch=%d, 1/2=%d, b/e=%d, steps=%d' % (n_epochs, n_batch, half_batch, bat_per_epo, n_steps))
#     # manually enumerate epochs
    
#     test_datagen = ImageDataGenerator()
    
#     if(with_aug):
#         train_datagen = ImageDataGenerator(
#                 rotation_range = 20,
#                 width_shift_range = 0.1,
#                 height_shift_range = 0.1,
#                 shear_range=0.2,
#                 zoom_range=0.2)
#     else:
#         train_datagen = ImageDataGenerator(
#                 horizontal_flip=False)
    
#     train_dgi = train_datagen.flow(X_sup, y_sup, n_batch)
#     X_all, y_all = dataset 
#     test_dgi = test_datagen.flow(X_all, y_all, 64)
#     print(test_dgi.n)
    
#     csv_logger = CSVLogger(folder +'/training.log')
#     check_point = ModelCheckpoint(folder+"/c_model_best.h5", monitor = "val_acc", verbose =1, save_best_only = True,mode = "max") 
    
#     c_model.fit_generator(train_dgi,
#             steps_per_epoch=bat_per_epo,
#             epochs=n_epochs,
#             validation_data = test_dgi,
#             validation_steps = test_dgi.n//(4*64),
#             callbacks = [csv_logger, check_point])

## size of the latent space

In [40]:
# size of the latent space
latent_dim = 100
#number of classes
n_classes=14
#number of labled samples
#arr_n_samples = [50,100,150,200,500,700,1000,2000,3000,4000,5000]
arr_n_samples = [3000]
 #batch sizes (for standalone classifier)
#n_batches = [8,8,8,16,16,32,32,32,32,64,64] 
n_batches = [120]  

stand_alone_flag = False #Change this for SGAN/STANDALONE
augmentation_flag = True #Change this for AUG/NO_AUG

## Assigning the dataset

In [41]:
dataset = images, labels

In [42]:
# dataset_val = images_val, labels_val

In [43]:
dataset2 = images_test, labels_test

## train model

In [44]:
if stand_alone_flag:
    for i in range(np.size(arr_n_samples ,0)):
        if augmentation_flag:
            folder = "testStandAlone_with_aug"+str(arr_n_samples[i])
        else:
            folder = "testStandAlone_without_aug"+str(arr_n_samples[i])
        if not os.path.exists("./"+folder+"/"):
                        os.makedirs("./"+folder+"/")
        
       
        # create the standalone classifier
        c_model = define_standalone_classifier(n_classes=n_classes)
    
        # load image data
        dataset = images, labels
        # train model
        n_batch = n_batches[i]
        train_stand_alone_c(c_model, dataset, folder, n_samples=arr_n_samples[i],n_epochs=200, n_batch=n_batch, n_classes=n_classes, with_aug=augmentation_flag)

else:   
    for i in range(np.size(arr_n_samples ,0)):
        if augmentation_flag:
            folder = "testSGAN_with_aug"+str(arr_n_samples[i])
        else:
            folder = "testSGAN_without_aug"+str(arr_n_samples[i]) 
        
        if not os.path.exists("./"+folder+"/"):
                        os.makedirs("./"+folder+"/")
        
        # create the discriminator models
        d_model, c_model = define_discriminator()
        # create the generator
        g_model = define_generator(latent_dim)
        # create the gan
        gan_model = define_gan(g_model, d_model)
        # load image data
        dataset = images, labels
        # train model
        train(g_model, d_model, c_model, gan_model, dataset, latent_dim, folder, n_samples=arr_n_samples[i], with_aug = augmentation_flag)

  0%|                                                                                           | 0/99 [00:00<?, ?it/s]

(11984, 120, 120, 3) (11984,)
n_epochs=50, n_batch=100, 1/2=50, b/e=99, steps=4950
Epoch 0/50


100%|██████████████████████████████████████████████████████████████████████████████████| 99/99 [23:54<00:00, 14.49s/it]


Best acc so far! best_acc_c = 21.489%
Best acc did not improve! best_acc_c = 21.489%


  0%|                                                                                           | 0/99 [00:00<?, ?it/s]

...

Best acc did not improve! best_acc_c = 21.489%
Best acc did not improve! best_acc_c = 21.489%


  1%|▊                                                                                  | 1/99 [00:00<00:14,  6.87it/s]

>Saved: testSGAN_with_aug3000/generated_plot_0002.png, testSGAN_with_aug3000/g_model_0002.h5, and testSGAN_with_aug3000/c_model_0002.h5
Epoch 2/50


  9%|███████▌                                                                           | 9/99 [00:46<07:44,  5.16s/it]


KeyboardInterrupt: 